We will plot the angle trace for the three longest whiskers (on each side if there are two sides)

In [ ]:
import os
# import cv2
import tables
from pathlib import Path
import numpy as np
import json
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Navigate to the directory where the video and tracking data is stored
# data_dir="E:/test/whisker_tracking"
data_dir=Path('/home/wanglab/data/whisker_asym/sc014/sc014_0325/sc014_0325_002/raw_data/behav_vid')
# video_name="whiskers.mp4"
video_name="sc014_0325_002_left.mp4" #"sc014_0325_002_TopCam0.mp4"
video_file=os.path.join(data_dir,video_name)
os.chdir(data_dir)

In [ ]:
# Open the HDF5 file and get the pixel data
h5file = tables.open_file(f'{os.path.basename(video_name).split(".")[0]}.hdf5', mode='r')

# Get the summary data
summary = h5file.get_node('/summary')

# Get unique whisker IDs
whisker_ids = np.unique(summary[:]['id'])


In [ ]:
summary

In [ ]:
# display first 10 values of summary time
summary[:10]['time']


In [ ]:
#  open the csv file data_dir / 'WT' / 'output_left.csv'
with open(data_dir / 'WT' / 'output_left.csv', 'r') as f:
    # read the first line of the file
    line = f.readline()
    # split the line into a list of strings
    line = line.split(',')
    # convert the strings to floats
    line = [float(x) for x in line]
    # print the first 10 values
    print(line[:10])

In [ ]:
# Get the table index for the first frame, with fid = 0 
first_frame_idx = np.where(summary[:]['fid'] == 0)[0]

# Get the index for the three longest whiskers in the first frame
longest_whiskers = np.argsort(summary[first_frame_idx]['length'])[-6:]

# Sort them by the follicle_y coordinate
longest_whiskers_idx = longest_whiskers[np.argsort(summary[first_frame_idx]['follicle_y'][longest_whiskers])]

# Get the wid for those whiskers
longest_whiskers_wid = summary[first_frame_idx]['wid'][longest_whiskers_idx]


In [ ]:
longest_whiskers_wid

In [ ]:
# Get the angle values for the longest whiskers

# Initialize l_angles as an empty list
l_angles = []
# Loop through the unique whisker IDs
for wid in longest_whiskers_wid:
    # Get the angle values for the current whisker ID
    w_angles = summary[summary[:]['wid'] == wid]['angle']
    # Append the angle values to the list of angles
    l_angles.append(w_angles)

# Second list comprehension: fill in the missing values with NaNs
# Get the length of the longest list of angles
max_len = max([len(l) for l in l_angles])
# Loop through the list of angles
for i in range(len(l_angles)):
    # Get the current list of angles
    w_angles = l_angles[i]
    # If the length of the current list of angles is less than the maximum length
    if len(w_angles) < max_len:
        # Append NaNs to the list of angles
        l_angles[i] = np.concatenate((w_angles, np.full(max_len - len(w_angles), np.nan)))

In [ ]:
l_angles

In [ ]:
# Create set of colors for those whiskers
colors = ['red', 'green', 'blue', 'orange', 'purple', 'brown']

# Plot the angle values for the longest whiskers, with different colors for each whisker
plt.figure(figsize=(20,10))
for i in range(len(l_angles)):
    plt.plot(l_angles[i], color=colors[i])
plt.xlabel('Time (frames)')
plt.ylabel('Angle (degrees)')
plt.title('Angle values for the longest whiskers')

In [ ]:
# Release the resources
h5file.close()